In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(Matrix)
    library(Seurat)
    library(R.utils)
})

ERROR: Error in library(Seurat): there is no package called ‘Seurat’


# Setting up data

Retrieve Seurat Object

In [ ]:
cd34_df <- readRDS(file = 'hultCD34_full.RDS')
cd34_df

See Shape of Object

In [ ]:
head(cd34_df)

In [ ]:
nrow(cd34_df)

In [ ]:
ncol(cd34_df)

Allign Usage scores with Seurat object

In [ ]:
cd34_df_alt <- subset(cd34_df, subset = nFeature_RNA >= 200)
cd34_df_alt

In [ ]:
DimPlot(cd34_df_alt, reduction = 'umap', group.by = 'subcluster')

In [ ]:
head(cd34_df_alt)

In [ ]:
cd34_metadata <-cd34_df_alt@meta.data
cd34_metadata

In [ ]:
nrow(cd34_df_alt)

In [ ]:
rownames(cd34_df_alt)

In [ ]:
ncol(cd34_df_alt)

In [ ]:
colnames(cd34_df_alt)

Import .CSV and .TXT

Contribution scores

In [ ]:
gcontri <- read.csv('cNMF4.gene_spectra_score.k_35.dt_0_15.csv')
head(gcontri)

In [ ]:
ncol(gcontri)

In [ ]:
nrow(gcontri)

Usage Scores (Matched with Seurat)

In [ ]:
gusage <- read.csv('cNMF4.usages.k_35.dt_0_15.consensus.txt', sep = "\t")
head(gusage)

In [ ]:
ncol(gusage)

In [ ]:
nrow(gusage)

Interesting GEPs
- GEP1 – Monocyte Lineage
- GEP4 – Elderly-biased HSC
- GEP6 – Late-Erythroid Lineage
- GEP14 – Basophil/Mast Cell Lineage
- GEP25 – Dendritic Cell Lineage
- GEP26 – Lymphoid Lineage
- GEP28 – Fetal-biased HSC
- GEP30 – Granulocyte Lineage
- GEP32 – Erythroid Lineage

*taken from cols*

In [ ]:
cluster_data <- cd34_metadata$cluster
unique(cluster_data)

In [ ]:
UMAPPlot(cd34_df_alt, group.by = "cluster", label = TRUE,
cols = c(
  "Uncommitted" = "grey",
  "Erythroid" = "#ee35358c",
  "Basophil/Mast" = "#0726d3cb",
  "Granulocyte" = "#f8f400b4",
  "Lymphoid" = "#0cc2fa",
  "Megakaryocyte" = "#7c0276",
  "Monocyte/DC" = "#06a85dcc"
)) + 
theme(legend.position = "none") +
labs(title = "UMAP Plot for GEPs")

# *CAT Analysis

Interesting GEPs
- GEP1 – Monocyte Lineage
- GEP4 – Elderly-biased HSC
- GEP6 – Late-Erythroid Lineage
- GEP14 – Basophil/Mast Cell Lineage
- GEP25 – Dendritic Cell Lineage
- GEP26 – Lymphoid Lineage
- GEP28 – Fetal-biased HSC
- GEP30 – Granulocyte Lineage
- GEP32 – Erythroid Lineage

*taken from cols*

Rename cd34_df_alt 

In [ ]:
cd34 <- cd34_df_alt

View Metadata

In [ ]:
cd34@meta.data %>% colnames

In [ ]:
length(cd34@assays$RNA@var.features)

In [ ]:
# counts = cd34@assays$RNA@counts

In [ ]:
# counts[1:5, 1:5]

In [ ]:
print(dim(cd34@assays$RNA@counts))

In [ ]:
# # Verify dimensions of the RNA assay's count matrix
# if (dim(cd34@assays$RNA@counts)[1] == 0 || dim(cd34@assays$RNA@counts)[2] == 0) {
#   stop("Error: The Seurat object's count matrix has zero features or zero cells.")
# } else {
#   message("Count matrix dimensions: ", dim(cd34@assays$RNA@counts)[1], " features by ", dim(cd34@assays$RNA@counts)[2], " cells.")
# }

# # Ensure variable features are selected correctly
# VariableFeatures(cd34) <- VariableFeatures(cd34)[VariableFeatures(cd34) %in% rownames(cd34@assays$RNA@counts)]

# # Check if any variable features remain after filtering
# if (length(VariableFeatures(cd34)) == 0) {
#   stop("Error: No valid variable features found. Check if FindVariableFeatures has been executed and returned features.")
# } else {
#   message("Number of variable features: ", length(VariableFeatures(cd34)))
# }

# # Proceed to scale the data, handling potential empty features gracefully
# tryCatch({
#   cd34 <- ScaleData(cd34, features = VariableFeatures(cd34))
#   message("Data scaling completed successfully.")
# }, error = function(e) {
#   message("Data scaling failed: ", e$message)
# })

In [ ]:
cd34@meta.data %>% colnames

In [ ]:
cd34@assays$RNA@var.features

In [ ]:
# counts_alt = cd34@assays$RNA@counts

In [ ]:
# dim(counts_alt)

In [ ]:
# counts_alt[1:5, 1:5]

In [ ]:
# dim(counts_alt)

In [ ]:
# length(barcodes)

In [ ]:
length(gene_names)

In [ ]:
# counts_fn

Try to convert csv to txt (correct file type)

In [ ]:
library(readr)

In [ ]:
write.table(gcontri, "cNMF4.gene_spectra_score.k_35.dt_0_15.txt", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

In [ ]:
gcontri_t <- as.data.frame(t(gcontri))

In [ ]:
ncol(gcontri_t)

In [ ]:
nrow(gcontri_t)

### DIMS mismatch

Gene Spectra Score: 
- dim: 36 x 40040
    - gene names: 40040
    - GEPs : 35

Matrix: 
- dim: 41569 x 57678
    - barcodes: 57678
    - gene names: 41569
- variable features: 2000

Less reference genes than Matrix

Try to find intersection. Courtesey of ChatGPT


In [ ]:
common_genes <- intersect(rownames(seurat_object), rownames(gcontri_t))

In [ ]:
cd34_int <- subset(cd34, features = common_genes)

Commented out after first run

In [ ]:
counts_int = cd34_int@assays$RNA@counts

In [ ]:
# Output counts matrix
writeMM(counts_int, 'matrix.mtx')
gzip('matrix.mtx')

# Output cell barcodes
barcodes <- colnames(counts_int)
write_delim(as.data.frame(barcodes), 'barcodes.tsv',
           col_names = FALSE)
gzip('barcodes.tsv')

# Output feature names
gene_names <- rownames(counts_int)
features <- data.frame("gene_id" = gene_names,"gene_name" = gene_names,type = "Gene Expression")
write_delim(as.data.frame(features),delim = "\t", 'features.tsv',
           col_names = FALSE)
gzip('features.tsv')

In [ ]:
data_dir ="./cd34hult/"
data_dir

In [ ]:
output_name = 'starCAT_cd34'
counts_fn = "matrix.mtx.gz"
gene_counts = "features.tsv.gz"

### Run *CAT

In [ ]:
# write.table(transpose_data, "cNMF4.gene_spectra_score.k_35.dt_0_15_t.txt", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

In [ ]:
cmd = paste0('starcat',
             ' --reference ', '"cNMF4.gene_spectra_score.k_35.dt_0_15.txt"',
             ' --counts ', '"', counts_fn, '"',
             ' --output-dir ', '"', data_dir, '"',
             ' --name ', '"', output_name, '"'
           )
cmd

In [ ]:
system(cmd)